In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # per definire il livello di log: non stampa info e warning, ma solo gli errori
import csv #modulo per i csv
from PIL import Image #image fornisce una classe per rappresentare un immagine pil. Pil è la libreria per la gestione delle immagini di python 
import numpy as np #fornisce le librerie come sklearn e scipy. Fornisce oggetti multidimensionali. Necessario per il ML
from sklearn.model_selection import train_test_split #sklearn libreria per ML. Fornisce modelli per classificazione, regressione, clustering, ecc. Train_test_split splitta gli array o le matrici in sottosezioni di train e test
from keras.preprocessing.image import img_to_array #keras è una libreria per ML. img_to_array converte una immagine PIL in un array Numpy
from keras.preprocessing.image import load_img # load_img carica una immagine nel formato PIL
from keras.models import Sequential #Sequential è una classe che fornisce le feature di training e inferenza 
from keras.layers import Dense, Conv2D, Flatten,Dropout, MaxPooling2D, AveragePooling2D 
# Dense layer: è il normale strato di rete neurale profondamente connesso. È il livello più comune e utilizzato di frequente. Il Dense Layer esegue l'operazione seguente sull'input e restituisce l'output.
# Conv2D layer: questo livello crea un kernel di convoluzione che è convoluto con l'input del livello per produrre un tensore di output. 
# Flatten layer: appiattisce l'input. 
# Dropout layer: imposta casualmente le unità di input su 0 con una frequenza di frequenza ad ogni passaggio durante il tempo di training, il che aiuta a prevenire il sovradattamento
# MaxPooling2D  layer: Operazione di pooling massimo per i dati spaziali 2D. Sottocampiona l'input lungo le sue dimensioni spaziali (altezza e larghezza) prendendo il valore massimo su una finestra di input (di dimensione definita da pool_size) per ogni canale dell'input. La finestra viene spostata di passi lungo ogni dimensione.
# AveragePooling2D layer: Sottocampiona l'input lungo le sue dimensioni spaziali (altezza e larghezza) prendendo il valore medio su una finestra di input (di dimensione definita da pool_size) per ogni canale dell'input. La finestra viene spostata di passi lungo ogni dimensione.
from sklearn.preprocessing import LabelBinarizer # Binarizza le etichette in modo uno contro tutti.In scikit-learn sono disponibili diversi algoritmi di regressione e di classificazione binaria. Un modo semplice per estendere questi algoritmi al caso di classificazione multiclasse consiste nell'utilizzare il cosiddetto schema uno contro tutti.Al momento dell'apprendimento, questo consiste semplicemente nell'imparare un regressore o un classificatore binario per classe. In tal modo, è necessario convertire le etichette multiclasse in etichette binarie (appartengono o non appartengono alla classe). LabelBinarizer semplifica questo processo con il metodo di trasformazione.Al momento della previsione, si assegna la classe per la quale il modello corrispondente ha dato la massima confidenza. LabelBinarizer lo rende facile con il metodo inverse_transform.
from tensorflow.python.keras.wrappers.scikit_learn import KerasClassifier # KerasClassifier: Implementazione dell'API del classificatore scikit-learn per Keras.
from sklearn.model_selection import RepeatedKFold, cross_val_score # RepeatedKFold ripete K-Fold n volte. Può essere utilizzato quando si richiede di eseguire KFold n volte, producendo split diversi in ogni ripetizione.
# cross_val_score valuta un punteggio tramite  cross-validation
import matplotlib.pyplot as plt #plotta i dati
from tensorflow.keras import regularizers

def open_image(X_value):
    return (1. / 255) * img_to_array(load_img('dataset/'+ X_value))


import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)




Num GPUs Available:  1


In [2]:
x_set = []
y_set = []

with open('index.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    for row in csv_reader:
        if len(row) > 1:
            if row[0] and row[1] and row[2] and row[3]:
                    if os.path.isfile("dataset/" + row[0]):
                        x_set.append(row[0])
                        y_set.append(row[3])

lb = LabelBinarizer()
y_set_cat = lb.fit_transform(y_set)
X_train, X_test, y_train, y_test = train_test_split(x_set, y_set_cat, test_size=0.33, random_state=1)

In [3]:
input_size = open_image(X_train[0]).shape
output_size = len(y_set_cat[0])

def baseline_model():
    model = Sequential()
    # The first two layers with 32 filters of window size 3x3
    model.add(Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=input_size, kernel_regularizer=regularizers.l2(l=0.01)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(64, (3, 3), padding='same', activation='relu', kernel_regularizer=regularizers.l2(l=0.01)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(64, (3, 3), padding='same', activation='relu', kernel_regularizer=regularizers.l2(l=0.01)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(500, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(output_size, activation='softmax'))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])

    return model
ann_model = KerasClassifier(build_fn=baseline_model, epochs=1, batch_size=16) #batch = quanti esempi prende per volta per aggiornare la rete neurale
fin_model = KerasClassifier(build_fn=baseline_model, epochs=1, batch_size=16)
X_train_adapt = np.array(list(map(open_image, X_train)))
baseline_model().summary()

In [ ]:
kfold= RepeatedKFold(n_splits=5, n_repeats=1)
results= cross_val_score(ann_model, X_train_adapt, y_train, cv=kfold, n_jobs=1, verbose=1)  # 2 cpus
results.mean()  # Mean MSE

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
2021-07-23 15:49:55.402299: E tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2021-07-23 15:49:55.402556: E tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


 89/416 [=====>........................] - ETA: 38s - loss: 8.0479 - acc: 0.1147

In [ ]:

history = fin_model.fit(X_train_adapt, y_train)

Error: Session cannot generate requests

In [ ]:
X_test_adapt = np.array(list(map(open_image, X_test)))
print(f' - Accuracy: {fin_model.score(X_test_adapt, y_test)}')

In [ ]:
plt.plot(history.history['acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [ ]:
y_pred = fin_model.predict(X_test)

def map_to_array(x): 
    final = [0] * len(lb.classes_)
    final[x] = 1
    return final
    
y_pred = np.array(list(map(map_to_array, list(y_pred))))
conf_matrix = confusion_matrix(y_test, lb.inverse_transform(y_pred), normalize='true', labels=accepted)
df_cm = pd.DataFrame(conf_matrix, index=accepted,
                     columns=accepted)
plt.figure(figsize=(10, 7))
sn.heatmap(df_cm, annot=True,cmap="Blues")
plt